In [1]:
import matplotlib.pyplot as plt
import numpy as np

from classiq import *
from classiq.applications.chemistry import (
    ChemistryExecutionParameters,
    HEAParameters,
    Molecule,
    MoleculeProblem,
    UCCParameters,
)
from classiq.execution import (
    ClassiqBackendPreferences,
    ClassiqSimulatorBackendNames,
    ExecutionPreferences,
    OptimizerType,
)

In [2]:
# https://docs.classiq.io/latest/explore/applications/chemistry/qpe_for_molecules/qpe_for_molecules/

molecule = Molecule(atoms=[("H", (0.0, 0.0, 0)), ("H", (0.0, 0.0, 0.735))])

In [3]:
# define your molecule problem

gs_problem = MoleculeProblem(
    molecule=molecule,
    basis="sto3g",
    mapping="jordan_wigner",  #'bravyi_kitaev'
    z2_symmetries=True,
    freeze_core=True,
)

operator = gs_problem.generate_hamiltonian()
gs_problem = gs_problem.update_problem(operator.num_qubits)
print("Your Hamiltonian is", operator.show(), sep="\n")

Your Hamiltonian is
-1.041 * I
-0.796 * Z
+0.181 * X


In [4]:
pauli_ops = operator.pauli_list
N = operator.num_qubits
print("Your Pauli list is", pauli_ops, sep="\n")

Your Pauli list is
[('I', (-1.0410931415166238+0j)), ('Z', (-0.7958748496863588+0j)), ('X', (0.18093119978423122+0j))]


In [5]:
# `n_qpe`-  The precision of the QPE (the number of qubits to be used in the QPE. Creates resolution of 1/2^n_qpe
n_qpe = 6

In [6]:
# recommanded n_qpe:
get_recommanded_n = False

import math

desired_resolution = 0.02


def get_nqpe(pauli_operator, desired_resolution):
    N = pauli_operator.num_qubits
    A = 0
    for a, b in pauli_operator.pauli_list:
        A = A + abs(b)
    nqpe = math.log2(2 * N * A / desired_resolution)
    return math.ceil(nqpe)


if get_recommanded_n:
    n_qpe = get_nqpe(operator, desired_resolution)
    print("number of qubits for QPE is", n_qpe)

In [7]:
from typing import cast

# normalizing the operator
## we need to create a matrix such that its normalized version will have eigenvalues of [0,1/2^k] when k is the resolution of the QPE
normalize = True


def normalization_params(pauli_list, N):
    A = 0
    for a, b in pauli_list:
        A = A + abs(b)
    return N * A


def normalize_hamiltonian(pauli_list, normalization_coeff, k):
    new_pauli_list = []
    for a, b in pauli_list:
        if a == "I" * N:
            new_pauli_list.append(
                (
                    a,
                    (b + normalization_coeff)
                    * (1 - 1 / (2**k))
                    / (2 * normalization_coeff),
                )
            )
        else:
            new_pauli_list.append((a, b * (1 - 1 / (2**k)) / (2 * normalization_coeff)))
    return new_pauli_list


pauli_list = pauli_ops
if normalize:
    normalization_coeff = normalization_params(pauli_ops, N)
    new_pauli_list = normalize_hamiltonian(pauli_ops, normalization_coeff, n_qpe)
    pauli_ops = new_pauli_list
    print(pauli_ops)

[('I', (0.23825359047723232+0j)), ('Z', (-0.19412250836394077+0j)), ('X', (0.04413108211329154+0j))]


In [8]:
CHAR_TO_STUCT_DICT = {"I": Pauli.I, "X": Pauli.X, "Y": Pauli.Y, "Z": Pauli.Z}


def pauli_str_to_enums(pauli):
    return [CHAR_TO_STUCT_DICT[s] for s in pauli]


def pauli_list_to_hamiltonian(pauli_list):
    return [
        PauliTerm(
            pauli=pauli_str_to_enums(pauli), coefficient=cast(complex, coeff).real
        )
        for pauli, coeff in pauli_list
    ]

In [9]:
from classiq.qmod.symbolic import log, pi

# this constant will be multipled be a linear factor for each qbit of the qpe, so the
# depth will scale linear with the power of each unitary, on the expense of approximating the
# exponentiation function
exp_max_depth = 2200


# define a Quantum Phase Estimation function that allows specifying each controlled
# power unitary directly
@qfunc
def my_flexible_qpe(
    unitary_with_power: QCallable[CInt],
    phase: QArray[QBit],
) -> None:
    apply_to_all(H, phase)

    repeat(
        count=phase.len,
        iteration=lambda index: control(
            ctrl=phase[index],
            stmt_block=lambda: unitary_with_power(2**index),
        ),
    )
    invert(lambda: qft(phase))


@qfunc
def main(reg: Output[QArray[QBit]], phase: Output[QNum]):
    allocate_num(num_qubits=n_qpe, is_signed=False, fraction_digits=n_qpe, out=phase)
    allocate(N, reg)

    # The state chosen for the initial state is HF, as usually it has good overlap with the with the eigenstate of minimal energy.
    molecule_hartree_fock(molecule_problem_to_qmod(gs_problem), reg)

    # # Hereby we send the pauli list created for the molecule for creating phase estimation circuit.
    my_flexible_qpe(
        lambda power: exponentiation_with_depth_constraint(
            pauli_list_to_hamiltonian(pauli_ops),
            evolution_coefficient=-2 * np.pi * power,
            max_depth=exp_max_depth * ((2**0.5) ** log(power, 2)),
            qbv=reg,
        ),
        phase,
    )

In [10]:
from classiq.execution import ExecutionPreferences

num_shots = 200000

qmod = create_model(
    main,
    constraints=Constraints(),  # feel free to fill in your constraints
    preferences=Preferences(timeout_seconds=600),
    execution_preferences=ExecutionPreferences(num_shots=num_shots),
    out_file="qpe_for_molecules",
)

In [11]:
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/2t8J0DS4hLskK7Ub05xDr4FShd5?version=0.68.0


In [12]:
result = execute(qprog).result_value()

In [13]:
result.parsed_counts_of_outputs("phase")[:10]

[{'phase': 0.046875}: 82150,
 {'phase': 0.03125}: 77774,
 {'phase': 0.0625}: 8969,
 {'phase': 0.015625}: 8834,
 {'phase': 0.0}: 3316,
 {'phase': 0.078125}: 3136,
 {'phase': 0.4375}: 2585,
 {'phase': 0.09375}: 1698,
 {'phase': 0.984375}: 1635,
 {'phase': 0.96875}: 988]

In [14]:
number_of_solutions = 1  # to be set

In [15]:
solutions = result.parsed_counts_of_outputs("phase")[:number_of_solutions]
print("Your qubit solution is", solutions, sep="\n")

Your qubit solution is
[{'phase': 0.046875}: 82150]


In [16]:
def post_process_phase(phase):
    return (
        (phase * 2 * normalization_coeff) / (1 - (1 / 2**n_qpe))
    ) - normalization_coeff

In [17]:
# renormalize into the "real" solution -
if normalize:
    solution = [post_process_phase(value.state["phase"]) for value in solutions]
else:
    solution = phase

In [18]:
print(solution)

[-1.8257183156550982]


In [19]:
if normalize:
    energy_resolution = (
        (1 / (2**n_qpe)) * 2 * normalization_coeff / (1 - (1 / 2**n_qpe))
    )
else:
    energy_resolution = 1 / (2**n_qpe)

print("the resolution of result is", energy_resolution)

for sol in solution:
    print(
        f"the eneregy solution is beteen {sol - energy_resolution} and {sol + energy_resolution}"
    )

    ###if zero or exceed the normalization range need to add conditions

the resolution of result is 0.06406029177737187
the eneregy solution is beteen -1.88977860743247 and -1.7616580238777264


In [20]:
mat = operator.to_matrix()
w, v = np.linalg.eig(mat)
print("exact result:", np.real(min(w)))

exact result: -1.8572750302023786


In [21]:
import matplotlib.pyplot as plt
import numpy as np

energy_vec = []
energy_prob = []

for value, counts in result.parsed_counts_of_outputs("phase"):
    temp = value[1]["phase"]
    if normalize:
        temp2 = (
            temp * 2 * normalization_coeff / (1 - (1 / 2**n_qpe))
        ) - normalization_coeff
    else:
        temp2 = temp
    energy_vec.append(temp2)
    energy_prob.append(counts[1] / num_shots)